In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from PIL import Image
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from ImageWork import *
import cv2

In [2]:
def pobitovo_sravnenie_WaterMark(W1, W2 , total_bit=1024 ):
    t1 = W1==W2
    sum=t1.sum()
    return sum/total_bit*100

def psnr(W, Wr):
    e = (np.sum((W - Wr) ** 2)) / (len(W) * len(W))
    p = 10 * np.log10(255 ** 2 / e)
    return p
    

def psnrCV2(W, Wr):
    p=cv2.PSNR(W,Wr)
    return p


def one_WM(DataFrame , path_save , path_water_mark , clf):
    del DataFrame['Unnamed: 0']
    X = DataFrame.iloc[:,[1,2,3,4]].values  
    Y=DataFrame.iloc[:,[0]].values
    y_predict_image=clf.predict(X)
    y_predict_image[y_predict_image>0]=255
    y_predict_image=y_predict_image.reshape(32,32)
    img = Image.fromarray(y_predict_image.astype(np.uint8))
    img.show()
    img.save(path_save)
    W=WaterMarkLoader.load(path_water_mark)
    WR=WaterMarkLoader.load(path_save)
    print("побитовое сравнение = ", pobitovo_sravnenie_WaterMark(W,WR))
    

def NO_AttackModel_Test(dataframe,path_save , title , clf, path_water_mark="Water Mark Image/WaterMarkRandom.jpg", index_image=1):
    X_dataframe = dataframe.iloc[:,[1,2,3,4]].values  
    Y_dataframe = dataframe.iloc[:,[0]].values
    X_train, X_test, y_train, y_test = train_test_split(X_dataframe, Y_dataframe, test_size=0.4, random_state=42)
    print(title)
    y_predict=clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    print(pd.DataFrame(cm))
    print()
    print(classification_report(y_test, y_predict))
    print()
    X_image_test = dataframe.iloc[:,[1,2,3,4]].values  
    i=index_image
    y=X_image_test[1024*i:1024*(i+1)]
    y_predict_image=clf.predict(y)
    y_predict_image[y_predict_image>0]=255
    y_predict_image=y_predict_image.reshape(32,32)
    img = Image.fromarray(y_predict_image.astype(np.uint8))
    #img.show()
    img.save(path_save)
    W=WaterMarkLoader.load(path_water_mark)
    WR=WaterMarkLoader.load(path_save)
    print("побитовое сравнение = ", pobitovo_sravnenie_WaterMark(W,WR))


In [10]:
No_Attack_frame = pd.read_csv('feature_vec/NO_Attack.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
del No_Attack_frame['Unnamed: 0']
X = No_Attack_frame.iloc[:,[1,2,3,4]].values  
Y=No_Attack_frame.iloc[:,[0]].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.35, random_state=42)

In [11]:
clf_no_attack = RandomForestClassifier (max_depth = 12, n_estimators = 50 ,verbose=True)
clf_no_attack.fit(X_train, y_train)

C:\Users\ANDREY~1\AppData\Local\Temp/ipykernel_12800/2407654023.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf_no_attack.fit(X_train, y_train)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    2.3s finished


RandomForestClassifier(max_depth=12, n_estimators=50, verbose=True)

In [12]:
y_predict=clf_no_attack.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predict)
print(pd.DataFrame(cm))
print(classification_report(y_test, y_predict))
#ПОСМОТРИ КАК ПРОГОНЯТЬ ДИАПОЗОЕ  ВЫСОТ И КОЛИЧЕСТВА УЗЛОВ

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


       0      1
0  18314    311
1    833  16382
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     18625
           1       0.98      0.95      0.97     17215

    accuracy                           0.97     35840
   macro avg       0.97      0.97      0.97     35840
weighted avg       0.97      0.97      0.97     35840



[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished


In [9]:
X_image_test = No_Attack_frame.iloc[:,[1,2,3,4]].values  
i=4
y=X_image_test[1024*i:1024*(i+1)]
y_predict_image=clf_no_attack.predict(y)
y_predict_image[y_predict_image>0]=255
y_predict_image=y_predict_image.reshape(32,32)
img = Image.fromarray(y_predict_image.astype(np.uint8))
img.show()
img.save("W_R/WR1.tif")
W=WaterMarkLoader.load("Water Mark Image/WaterMarkRandom.jpg")
WR=WaterMarkLoader.load("W_R/WR1.tif")
print("побитовое сравнение = ", pobitovo_sravnenie_WaterMark(W,WR))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


побитовое сравнение =  57.91015625


In [33]:
path_averege= "feature_vec/AverageAttack.txt"
path_histogram="feature_vec/HistogramAttack.txt"
path_GammaCor=  "feature_vec/GammaCorrection.txt"
path_JPEG50= "feature_vec/JPEG50.txt"
path_median=  "feature_vec/medianAttack.txt"
path_saltpaper="feature_vec/SaltPaperAttack.txt"
path_snarpness= "feature_vec/Sharpness.txt"

In [34]:
averege=pd.read_csv(path_averege) 
histogram=pd.read_csv(path_histogram)
GammaCor=pd.read_csv(path_GammaCor) 
JPEG50=pd.read_csv(path_JPEG50) 
median=pd.read_csv(path_median) 
saltpaper=pd.read_csv(path_saltpaper)
snarpness=pd.read_csv(path_snarpness) 
frames = [averege, histogram, GammaCor , JPEG50, median,saltpaper ,snarpness]
DataFrame = pd.concat(frames)
del DataFrame['Unnamed: 0']
del saltpaper['Unnamed: 0']
del averege['Unnamed: 0']
del histogram['Unnamed: 0']
del GammaCor['Unnamed: 0']
del median['Unnamed: 0']
del JPEG50['Unnamed: 0']
del snarpness['Unnamed: 0']

DataFrame

,bit,feature_1,feature_2,feature_3,feature_4
0,1,-0.5,0.0,0.0,0.0
1,0,0.5,-0.5,0.5,-0.5
2,1,2.5,0.0,0.5,3.5
3,1,-1.0,-1.5,3.0,-4.0
4,1,-0.5,1.0,-2.5,-1.0
...,...,...,...,...,...
204795,0,-25.0,-11.0,-12.5,-13.5
204796,1,8.5,1.5,20.5,5.0
204797,1,-3.5,11.5,-5.5,-5.0
204798,1,-1.0,19.5,0.0,-6.0


### тестирую модель обученную на noattack подсовываю атакованные изображения

In [22]:
NO_AttackModel_Test(saltpaper,"W_R/WR_SP.tif", "ПРОВЕРКА ДЛЯ АТАКИ СОЛЬ-ПЕРЕЦ. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack,)

ПРОВЕРКА ДЛЯ АТАКИ СОЛЬ-ПЕРЕЦ. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


       0      1
0  38988   3396
1   1055  38481

              precision    recall  f1-score   support

           0       0.97      0.92      0.95     42384
           1       0.92      0.97      0.95     39536

    accuracy                           0.95     81920
   macro avg       0.95      0.95      0.95     81920
weighted avg       0.95      0.95      0.95     81920


побитовое сравнение =  95.41015625


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


In [13]:
Attack_Dota_SP = pd.read_csv('feature_vec/SaltPaperAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_SP ,"W_R/WR_SP_Dota.tif","Water Mark Image/waterMark3.jpg" , clf_no_attack )


FileNotFoundError: [Errno 2] No such file or directory: 'feature_vec/SaltPaperAttackDota.txt'

In [35]:
NO_AttackModel_Test(JPEG50,"W_R/WR_JPEG50.tif", "ПРОВЕРКА ДЛЯ АТАКИ JPEG50. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

ПРОВЕРКА ДЛЯ АТАКИ JPEG50. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


       0     1
0  37990  4394
1  35163  4373

              precision    recall  f1-score   support

           0       0.52      0.90      0.66     42384
           1       0.50      0.11      0.18     39536

    accuracy                           0.52     81920
   macro avg       0.51      0.50      0.42     81920
weighted avg       0.51      0.52      0.43     81920


побитовое сравнение =  50.9765625


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


In [14]:
Attack_Dota_jpeg = pd.read_csv('feature_vec/JPEG50Dota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_jpeg ,"W_R/WR_jpeg_Dota.tif","Water Mark Image/waterMark3.jpg" ,clf_no_attack )

побитовое сравнение =  54.6875


In [36]:
NO_AttackModel_Test(averege,"W_R/WR_averege.tif", "ПРОВЕРКА ДЛЯ АТАКИ averege - среднее. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

ПРОВЕРКА ДЛЯ АТАКИ averege - среднее. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


       0    1
0  41518  866
1  38684  852

              precision    recall  f1-score   support

           0       0.52      0.98      0.68     42384
           1       0.50      0.02      0.04     39536

    accuracy                           0.52     81920
   macro avg       0.51      0.50      0.36     81920
weighted avg       0.51      0.52      0.37     81920


побитовое сравнение =  51.5625


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


In [17]:
Attack_Dota_Average = pd.read_csv('feature_vec/AverageAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_Average ,"W_R/WR_Average_Dota.tif","Water Mark Image/waterMark3.jpg",clf_no_attack )

побитовое сравнение =  54.39453125


In [37]:
NO_AttackModel_Test(histogram,"W_R/WR_histogram.tif", "ПРОВЕРКА ДЛЯ АТАКИ histogram. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

ПРОВЕРКА ДЛЯ АТАКИ histogram. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


       0      1
0  38187   4197
1   8502  31034

              precision    recall  f1-score   support

           0       0.82      0.90      0.86     42384
           1       0.88      0.78      0.83     39536

    accuracy                           0.84     81920
   macro avg       0.85      0.84      0.84     81920
weighted avg       0.85      0.84      0.84     81920


побитовое сравнение =  94.04296875


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


In [19]:
Attack_Dota_Histogram = pd.read_csv('feature_vec/HistogramAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_Histogram ,"W_R/WR_Histogram_Dota.tif","Water Mark Image/waterMark3.jpg",clf_no_attack )

побитовое сравнение =  82.32421875


In [38]:
NO_AttackModel_Test(GammaCor,"W_R/WR_GammaCor.tif", "ПРОВЕРКА ДЛЯ АТАКИ GammaCor. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

ПРОВЕРКА ДЛЯ АТАКИ GammaCor. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


       0      1
0  41834    550
1   7368  32168

              precision    recall  f1-score   support

           0       0.85      0.99      0.91     42384
           1       0.98      0.81      0.89     39536

    accuracy                           0.90     81920
   macro avg       0.92      0.90      0.90     81920
weighted avg       0.91      0.90      0.90     81920


побитовое сравнение =  83.49609375


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


In [21]:
Attack_Dota_GammaCorrection= pd.read_csv('feature_vec/GammaCorrectionCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_GammaCorrection ,"W_R/WR_GammaCorrection.tif","Water Mark Image/waterMark3.jpg",clf_no_attack )

побитовое сравнение =  53.515625


In [39]:
NO_AttackModel_Test(median,"W_R/WR_median.tif", "ПРОВЕРКА ДЛЯ АТАКИ median. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

ПРОВЕРКА ДЛЯ АТАКИ median. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


       0     1
0  38997  3387
1  35072  4464

              precision    recall  f1-score   support

           0       0.53      0.92      0.67     42384
           1       0.57      0.11      0.19     39536

    accuracy                           0.53     81920
   macro avg       0.55      0.52      0.43     81920
weighted avg       0.55      0.53      0.44     81920


побитовое сравнение =  53.3203125


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


In [25]:
Attack_Cross_medianAttack= pd.read_csv('feature_vec/medianAttackCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Cross_medianAttack ,"W_R/medianAttackCross.tif","Water Mark Image/waterMark2.jpg",clf_no_attack )

побитовое сравнение =  64.16015625


In [40]:
NO_AttackModel_Test(snarpness,"W_R/WR_snarpness.tif", "ПРОВЕРКА ДЛЯ АТАКИ snarpness. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

ПРОВЕРКА ДЛЯ АТАКИ snarpness. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


       0      1
0  40690   1694
1    217  39319

              precision    recall  f1-score   support

           0       0.99      0.96      0.98     42384
           1       0.96      0.99      0.98     39536

    accuracy                           0.98     81920
   macro avg       0.98      0.98      0.98     81920
weighted avg       0.98      0.98      0.98     81920


побитовое сравнение =  97.8515625


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


In [28]:
Attack_Cross_Sharpness= pd.read_csv('feature_vec/SharpnessCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Cross_Sharpness ,"W_R/SharpnessCross.tif","Water Mark Image/waterMark2.jpg",clf_no_attack )

побитовое сравнение =  75.1953125


### тестирую модель обученную на attack image подсовываю атакованные изображения

In [42]:
X = DataFrame.iloc[:,[1,2,3,4]].values  
Y=DataFrame.iloc[:,[0]].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

In [43]:
clf_attack = RandomForestClassifier (max_depth = 12, n_estimators = 50 ,verbose=True)
clf_attack.fit(X_train, y_train)

C:\Users\ANDREY~1\AppData\Local\Temp/ipykernel_8688/2247406573.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf_attack.fit(X_train, y_train)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.3min finished


RandomForestClassifier(max_depth=12, n_estimators=50, verbose=True)

In [45]:
y_predict=clf_attack.predict(X_test)
cm = confusion_matrix(y_test, y_predict)
print(pd.DataFrame(cm))
print(classification_report(y_test, y_predict))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    3.9s finished


        0       1
0  279967   15962
1  117154  160357
              precision    recall  f1-score   support

           0       0.70      0.95      0.81    295929
           1       0.91      0.58      0.71    277511

    accuracy                           0.77    573440
   macro avg       0.81      0.76      0.76    573440
weighted avg       0.80      0.77      0.76    573440



In [46]:
NO_AttackModel_Test(saltpaper,"W_R/WR_SP.tif", "ПРОВЕРКА ДЛЯ АТАКИ СОЛЬ-ПЕРЕЦ. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

ПРОВЕРКА ДЛЯ АТАКИ СОЛЬ-ПЕРЕЦ. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


       0      1
0  40978   1406
1   1375  38161

              precision    recall  f1-score   support

           0       0.97      0.97      0.97     42384
           1       0.96      0.97      0.96     39536

    accuracy                           0.97     81920
   macro avg       0.97      0.97      0.97     81920
weighted avg       0.97      0.97      0.97     81920


побитовое сравнение =  96.875


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


In [34]:
Attack_Dota_SP = pd.read_csv('feature_vec/SaltPaperAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_SP ,"W_R/WR_SP_Dota.tif","Water Mark Image/waterMark3.jpg",clf_attack )


побитовое сравнение =  84.27734375


In [47]:
NO_AttackModel_Test(averege,"W_R/WR_averege.tif", "ПРОВЕРКА ДЛЯ АТАКИ averege - среднее. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

ПРОВЕРКА ДЛЯ АТАКИ averege - среднее. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


       0     1
0  41111  1273
1  38198  1338

              precision    recall  f1-score   support

           0       0.52      0.97      0.68     42384
           1       0.51      0.03      0.06     39536

    accuracy                           0.52     81920
   macro avg       0.52      0.50      0.37     81920
weighted avg       0.52      0.52      0.38     81920


побитовое сравнение =  51.46484375


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


In [38]:
Attack_Dota_Average = pd.read_csv('feature_vec/AverageAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_Average ,"W_R/WR_Average_Dota.tif","Water Mark Image/waterMark3.jpg",clf_attack )

побитовое сравнение =  51.7578125


In [48]:
NO_AttackModel_Test(histogram,"W_R/WR_histogram.tif", "ПРОВЕРКА ДЛЯ АТАКИ histogram. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

ПРОВЕРКА ДЛЯ АТАКИ histogram. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


       0      1
0  38913   3471
1   6567  32969

              precision    recall  f1-score   support

           0       0.86      0.92      0.89     42384
           1       0.90      0.83      0.87     39536

    accuracy                           0.88     81920
   macro avg       0.88      0.88      0.88     81920
weighted avg       0.88      0.88      0.88     81920


побитовое сравнение =  96.484375


In [40]:
Attack_Dota_Histogram = pd.read_csv('feature_vec/HistogramAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_Histogram ,"W_R/WR_Histogram_Dota.tif","Water Mark Image/waterMark3.jpg",clf_attack )

побитовое сравнение =  84.86328125


In [49]:
NO_AttackModel_Test(GammaCor,"W_R/WR_GammaCor.tif", "ПРОВЕРКА ДЛЯ АТАКИ GammaCor. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

ПРОВЕРКА ДЛЯ АТАКИ GammaCor. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


       0      1
0  42162    222
1    945  38591

              precision    recall  f1-score   support

           0       0.98      0.99      0.99     42384
           1       0.99      0.98      0.99     39536

    accuracy                           0.99     81920
   macro avg       0.99      0.99      0.99     81920
weighted avg       0.99      0.99      0.99     81920


побитовое сравнение =  98.73046875


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


In [42]:
Attack_Dota_GammaCorrection= pd.read_csv('feature_vec/GammaCorrectionCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_GammaCorrection ,"W_R/WR_GammaCorrection.tif","Water Mark Image/waterMark3.jpg",clf_attack )

побитовое сравнение =  49.21875


In [27]:
NO_AttackModel_Test(median,"W_R/WR_median.tif", "ПРОВЕРКА ДЛЯ АТАКИ median. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

ПРОВЕРКА ДЛЯ АТАКИ median. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  34550   7834
1  28211  11325

              precision    recall  f1-score   support

           0       0.55      0.82      0.66     42384
           1       0.59      0.29      0.39     39536

    accuracy                           0.56     81920
   macro avg       0.57      0.55      0.52     81920
weighted avg       0.57      0.56      0.53     81920


побитовое сравнение =  55.76171875


In [44]:
Attack_Cross_medianAttack= pd.read_csv('feature_vec/medianAttackCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Cross_medianAttack ,"W_R/medianAttackCross.tif","Water Mark Image/waterMark2.jpg",clf_attack )

побитовое сравнение =  57.71484375


In [50]:
NO_AttackModel_Test(snarpness,"W_R/WR_snarpness.tif", "ПРОВЕРКА ДЛЯ АТАКИ snarpness. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

ПРОВЕРКА ДЛЯ АТАКИ snarpness. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


       0      1
0  41752    632
1    186  39350

              precision    recall  f1-score   support

           0       1.00      0.99      0.99     42384
           1       0.98      1.00      0.99     39536

    accuracy                           0.99     81920
   macro avg       0.99      0.99      0.99     81920
weighted avg       0.99      0.99      0.99     81920


побитовое сравнение =  99.31640625


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


In [46]:
Attack_Cross_Sharpness= pd.read_csv('feature_vec/SharpnessCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Cross_Sharpness ,"W_R/SharpnessCross.tif","Water Mark Image/waterMark2.jpg",clf_attack )

побитовое сравнение =  78.3203125


In [51]:
NO_AttackModel_Test(JPEG50,"W_R/WR_JPEG50.tif", "ПРОВЕРКА ДЛЯ АТАКИ JPEG50. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

ПРОВЕРКА ДЛЯ АТАКИ JPEG50. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


       0     1
0  38182  4202
1  35084  4452

              precision    recall  f1-score   support

           0       0.52      0.90      0.66     42384
           1       0.51      0.11      0.18     39536

    accuracy                           0.52     81920
   macro avg       0.52      0.51      0.42     81920
weighted avg       0.52      0.52      0.43     81920


побитовое сравнение =  52.5390625


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


In [48]:
Attack_Dota_jpeg = pd.read_csv('feature_vec/JPEG50Dota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_jpeg ,"W_R/WR_jpeg_Dota.tif","Water Mark Image/waterMark3.jpg" ,clf_attack )

побитовое сравнение =  54.98046875


#  Обучаю сеть на одной конкретной атаке и смотрю на результат

### Медианная фильтрация

In [30]:
median=pd.read_csv(path_median) 
No_Attack_frame = pd.read_csv('feature_vec/NO_Attack.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
frames = [ median , No_Attack_frame ]
DataFrame = pd.concat(frames)
del DataFrame['Unnamed: 0']
del median['Unnamed: 0']
DataFrame

,bit,feature_1,feature_2,feature_3,feature_4
0,1,0.0,0.5,-1.5,1.5
1,0,-3.0,-31.5,-5.0,-38.0
2,1,0.0,29.0,0.5,29.5
3,1,-31.0,-4.0,-44.0,4.5
4,1,-21.0,0.0,-17.0,-3.5
...,...,...,...,...,...
204795,0,-6.5,3.0,-1.0,3.0
204796,1,-3.5,11.5,0.5,-2.5
204797,1,0.5,-8.0,11.5,-2.5
204798,1,2.0,28.0,-2.0,45.0


In [31]:
X = DataFrame.iloc[:,[1,2,3,4]].values  
Y=DataFrame.iloc[:,[0]].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.35, random_state=42)

In [32]:
clf_specific_attack = MLPClassifier(hidden_layer_sizes = (20,15,10,5) , max_iter=1000 ,early_stopping=True , verbose=True)
clf_specific_attack.fit(X_train, y_train)

d:\code\python\nir\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.59281706
Validation score: 0.756761
Iteration 2, loss = 0.52534694
Validation score: 0.774076
Iteration 3, loss = 0.51036256
Validation score: 0.772048
Iteration 4, loss = 0.50160991
Validation score: 0.775503
Iteration 5, loss = 0.49453901
Validation score: 0.776367
Iteration 6, loss = 0.48927642
Validation score: 0.777231
Iteration 7, loss = 0.48560861
Validation score: 0.777306
Iteration 8, loss = 0.48330229
Validation score: 0.778621
Iteration 9, loss = 0.48101394
Validation score: 0.779184
Iteration 10, loss = 0.47940130
Validation score: 0.780311
Iteration 11, loss = 0.47789233
Validation score: 0.780048
Iteration 12, loss = 0.47704731
Validation score: 0.779973
Iteration 13, loss = 0.47611274
Validation score: 0.779785
Iteration 14, loss = 0.47551980
Validation score: 0.778583
Iteration 15, loss = 0.47501093
Validation score: 0.781025
Iteration 16, loss = 0.47426552
Validation score: 0.779973
Iteration 17, loss = 0.47349904
Validation score: 0.779935
Iterat

MLPClassifier(early_stopping=True, hidden_layer_sizes=(20, 15, 10, 5),
              max_iter=1000, verbose=True)

In [33]:
NO_AttackModel_Test(median,"W_R/WR_median.tif", "ПРОВЕРКА ДЛЯ АТАКИ median. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_specific_attack)

ПРОВЕРКА ДЛЯ АТАКИ median. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  30812  11572
1  23816  15720

              precision    recall  f1-score   support

           0       0.56      0.73      0.64     42384
           1       0.58      0.40      0.47     39536

    accuracy                           0.57     81920
   macro avg       0.57      0.56      0.55     81920
weighted avg       0.57      0.57      0.56     81920


побитовое сравнение =  56.34765625


In [65]:
Attack_Cross_medianAttack= pd.read_csv('feature_vec/medianAttackCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Cross_medianAttack ,"W_R/medianAttackCross.tif","Water Mark Image/waterMark2.jpg",clf_specific_attack )

побитовое сравнение =  62.6953125


### Среднее

In [34]:
averege=pd.read_csv(path_averege) 
No_Attack_frame = pd.read_csv('feature_vec/NO_Attack.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
frames = [ averege , No_Attack_frame ]
DataFrame = pd.concat(frames)
del DataFrame['Unnamed: 0']
del averege['Unnamed: 0']
DataFrame

X = DataFrame.iloc[:,[1,2,3,4]].values  
Y=DataFrame.iloc[:,[0]].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.35, random_state=42)

clf_specific_attack = MLPClassifier(hidden_layer_sizes = (20,15,10,5) , max_iter=1000 ,early_stopping=True , verbose=True)
clf_specific_attack.fit(X_train, y_train)

d:\code\python\nir\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.59898790
Validation score: 0.748235
Iteration 2, loss = 0.51925582
Validation score: 0.752779
Iteration 3, loss = 0.50569332
Validation score: 0.757399
Iteration 4, loss = 0.50031389
Validation score: 0.757812
Iteration 5, loss = 0.49624868
Validation score: 0.758676
Iteration 6, loss = 0.49306574
Validation score: 0.759578
Iteration 7, loss = 0.49029154
Validation score: 0.760554
Iteration 8, loss = 0.48819053
Validation score: 0.760705
Iteration 9, loss = 0.48638716
Validation score: 0.759991
Iteration 10, loss = 0.48492682
Validation score: 0.760817
Iteration 11, loss = 0.48356299
Validation score: 0.761306
Iteration 12, loss = 0.48264327
Validation score: 0.760254
Iteration 13, loss = 0.48155514
Validation score: 0.760367
Iteration 14, loss = 0.48088148
Validation score: 0.761306
Iteration 15, loss = 0.47967852
Validation score: 0.761756
Iteration 16, loss = 0.47872190
Validation score: 0.761719
Iteration 17, loss = 0.47809091
Validation score: 0.761944
Iterat

MLPClassifier(early_stopping=True, hidden_layer_sizes=(20, 15, 10, 5),
              max_iter=1000, verbose=True)

In [35]:
NO_AttackModel_Test(median,"W_R/WR_average.tif", "ПРОВЕРКА ДЛЯ АТАКИ average МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_specific_attack)

ПРОВЕРКА ДЛЯ АТАКИ average МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ
       0      1
0  31144  11240
1  24393  15143

              precision    recall  f1-score   support

           0       0.56      0.73      0.64     42384
           1       0.57      0.38      0.46     39536

    accuracy                           0.57     81920
   macro avg       0.57      0.56      0.55     81920
weighted avg       0.57      0.57      0.55     81920


побитовое сравнение =  57.03125


In [36]:
Attack_Dota_Average = pd.read_csv('feature_vec/AverageAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_Average ,"W_R/WR_Average_Dota.tif","Water Mark Image/waterMark3.jpg",clf_specific_attack )

FileNotFoundError: [Errno 2] No such file or directory: 'feature_vec/AverageAttackDota.txt'